# Лабораторная работа №6: Реализация голосового ввода для управления функциями информационных систем

---
---

Задание:
1. Необходимо реализовать модуль, который записывает команду голосом с микрофона по умолчанию, превращает ее в текст, извлекает из текста ключевые слова, если находит в ключевых словах команду, выполняет ее.
2. Для этого нужно создать виртуальную среду  `python -m venv ./venv`
3. Установить библиотеку для записи аудио `pyaudio`: `python -m pip install pyaudio` ([PyAudio: Cross-platform audio I/O for Python, with PortAudio](https://people.csail.mit.edu/hubert/pyaudio/#docs))
4. Установить платформу для распознавания речи Whisper ([GitHub - openai/whisper: Robust Speech Recognition via Large-Scale Weak Supervision](https://github.com/openai/whisper)): `pip install git+https://github.com/openai/whisper.git`
5. Воспользовавшись наработками из лабораторной работы №4, реализовать препроцессинг голосовой команды и извлечение ключевых слов для вариантов ниже:
    1. для генерации ссылки на поиск, например, в Яндексе произвольного текста в браузере по умолчанию
    2. для открытия видеофайла из файловой системы в плеере по умолчанию или поиска видео на YouTube (или аналогичной платформе)
    3. для запуска любого приложения для коммуникации (ВК, Телеграм, Дискорд и т.п.)
6. Реализовать через `subprocess` или иные средства выполнение указанных команд

>[!note]
>Передавать в Whisper можно как .wav-файл, так и NumPy-массив
>Но тут есть нюансы, поэтому рекомендую прочесть эти две ссылки: 
>- [How to send audio to Whisper in a numpy array ? · openai/whisper · Discussion #450 · GitHub](https://github.com/openai/whisper/discussions/450)
>- [python - How to feed a numpy array as audio for whisper model - Stack Overflow](https://stackoverflow.com/questions/76448210/how-to-feed-a-numpy-array-as-audio-for-whisper-model)


---
---

In [1]:
import wave
import sys
import pyaudio
import os
import subprocess
import webbrowser


CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1 if sys.platform == 'darwin' else 2
RATE = 44100
RECORD_SECONDS = 5

try:   
    with wave.open('output.wav', 'wb') as wf:
        p = pyaudio.PyAudio()
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)

        stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True)

        print('Recording...')
        for _ in range(0, RATE // CHUNK * RECORD_SECONDS):
            wf.writeframes(stream.read(CHUNK))
        print('Done')

        stream.close()
        p.terminate()
except:
    print('error')